In [26]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import math
import sys
import os
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

In [27]:
class DecisionStump():
    def __init__(self):
        self.polarity = 1
        self.feature_index = None
        self.threshold = None
        self.alpha = None

In [28]:
class AdaBoost():
    def __init__(self, n_classifier=5):
        self.n_classifier = n_classifier
        # List of weak classifiers
        self.classifiers = []
 
    def fit(self, X, y):
 
        n_samples, n_features = np.shape(X)
 
        w = np.full(n_samples, (1 / n_samples))
         
        for _ in range(self.n_classifier):
            classifier = DecisionStump()
            min_error = 1
            for feature_i in range(n_features):
                feature_values = np.expand_dims(X[:, feature_i], axis=1)
                unique_values = np.unique(feature_values)
                for threshold in unique_values:
                    p = 1
                    prediction = np.ones(np.shape(y))
                    prediction[X[:, feature_i] < threshold] = -1
                    error = sum(w[y != prediction])
                     
                    if error > 0.5:
                        error = 1 - error
                        p = -1
 
                    if error < min_error:
                        classifier.polarity = p
                        classifier.threshold = threshold
                        classifier.feature_index = feature_i
                        min_error = error
            
            classifier.alpha = 0.5 * math.log((1.0 - min_error) / (min_error + 1e-10))
 
            predictions = np.ones(np.shape(y))
            negative_index = (classifier.polarity * X[:, classifier.feature_index] < classifier.polarity * classifier.threshold)
            predictions[negative_index] = -1
 
            w = w.dot(np.exp(classifier.alpha * y.dot(predictions)))
            w /= np.sum(w)
            self.classifiers.append(classifier)
 
    def predict(self, X):
        n_samples = np.shape(X)[0]
        y_predicted = np.zeros((n_samples, 1))
        for classifier in self.classifiers:
            predictions = np.ones(np.shape(y_predicted))
            negative_index = (classifier.polarity * X[:, classifier.feature_index] < classifier.polarity * classifier.threshold)
            predictions[negative_index] = -1
            y_predicted = np.concatenate((y_predicted, classifier.alpha * predictions), axis=1)
        y_predicted = np.sign(np.sum(y_predicted, axis=1))
 
        return y_predicted

In [29]:
def main():
    data = datasets.load_iris()
    X = data.data
    y = data.target
 
    X = X[y != 2]
    y = y[y != 2]
    y[y == 0] = -1
    y[y == 1] = 1
 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
 
    classifier = AdaBoost(n_classifier=10)
    classifier.fit(X_train, y_train)
    y_predicted = classifier.predict(X_test)
 
    accuracy = accuracy_score(y_test, y_predicted)
 
    print ("Accuracy:", accuracy)
  
if __name__ == "__main__":
    main()

Accuracy: 0.98
